This is the inference kernel for BERT pytorch. Check out the amazing Kernel for finetuning BERT by Yuval: https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila

In [1]:
import os
os.listdir('E:/bertmodel/Bert Pretrained Models/uncased_l-12_h-768_a-12/')

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'bert_model.ckpt.meta',
 'vocab.txt']

In [2]:
import sys
package_dir = "E:/bertmodel//pytorch-pretrained-BERT"
sys.path.append(package_dir)

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig

warnings.filterwarnings(action='once')
device = torch.device('cuda')

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [18]:
new = pd.read_csv('train.csv')

In [19]:
new.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,26650,approved,0,0,0,1,0,0.000000,0,4


In [27]:
new.loc[new['target'] >= 0.5].head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,val
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,rejected,0,0,0,1,0,0.000000,4,47,59856
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0000,NaN,NaN,...,rejected,0,0,0,0,0,0.009524,0,105,59859
13,239583,0.600000,It's ridiculous that these guys are being call...,0.000000,0.100000,0.000000,0.600000,0.1000,NaN,NaN,...,approved,0,0,0,3,0,0.000000,0,10,239583
14,239584,0.500000,This story gets more ridiculous by the hour! A...,0.000000,0.000000,0.000000,0.300000,0.0000,NaN,NaN,...,approved,0,0,0,9,0,0.000000,0,10,239584
19,239592,0.500000,"Angry trolls, misogynists and Racists"", oh my....",0.000000,0.000000,0.100000,0.500000,0.0000,0.0,0.0,...,approved,0,0,0,0,0,0.000000,4,10,239592
31,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,approved,0,0,0,1,0,0.000000,4,80,239607
34,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,rejected,0,0,0,0,0,0.061538,4,65,239612
84,239699,0.500000,"Tuesday Jan. 13, members of \nthis socalled 'm...",0.000000,0.000000,0.000000,0.500000,0.0000,NaN,NaN,...,approved,0,0,0,3,0,0.000000,0,10,239699
105,239731,0.600000,Love that they also participate in the Portlan...,0.000000,0.000000,0.000000,0.600000,0.0000,NaN,NaN,...,approved,0,0,0,1,0,0.000000,0,10,239731
123,239762,0.700000,Took this as an opportunity to check back in o...,0.100000,0.000000,0.000000,0.700000,0.0000,NaN,NaN,...,approved,0,0,0,0,0,0.000000,0,10,239762


In [4]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [23]:
BERT_MODEL_PATH = 'E:/bertmodel/Bert Pretrained Models/uncased_l-12_h-768_a-12/'
bert_config = BertConfig('E:/bertmodel/bert_inference/bert_config.json')
model.load_state_dict(torch.load("E:/bertmodel/bert_inference/bert_pytorch.bin"))


In [5]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
BATCH_SIZE = 32
BERT_MODEL_PATH = 'E:/bertmodel/Bert Pretrained Models/uncased_l-12_h-768_a-12/'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

bert_config = BertConfig('E:/bertmodel/bert_inference/bert_config.json')
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [6]:
test_df = pd.read_csv("test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

100%|███████████████████████████████████████████████████████████████████████████| 97320/97320 [03:18<00:00, 489.77it/s]


In [8]:
model = BertForSequenceClassification(bert_config, num_labels=1)
model.load_state_dict(torch.load("E:/bertmodel/bert_inference/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme

In [9]:
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

  1%|█                                                                             | 43/3042 [02:51<3:16:47,  3.94s/it]c:\users\beyou\appdata\local\programs\python\python36\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\beyou\appdata\local\programs\python\python36\lib\site-packages\sphinxcontrib: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
c:\users\beyou\appdata\local\programs\python\python36\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\beyou\appdata\local\programs\python\python36\lib\site-packages\google: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
c:\users\beyou\appdata\local\programs\python\python36\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\beyou\appdata\local\programs\python\python36\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
c

KeyboardInterrupt: 

In [ ]:
submission = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred
})
submission.to_csv('output.csv', index=False)

In [ ]:
submission.head()